<img src="images/dwave_leap.png" width="500 px" align="center">

# Solving Optimization Problems on DWAVE





Optimization in general is to find the best choice among a set of options, subject to a set of constraints. Formulated in words

<div align=center>
optimize <strong>objective</strong><br>
by varying <strong>variables</strong><br>
subject to <strong>constraints</strong>
</div>
<br>

We can use D-Wave to solve these kind of problems if the objective and constraints are quadratic at most and the variables have binary values.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# How to build a QUBO - Part 2

Let us consider the following optimization problem with constraints:
    
**Problem**: 

You have the following set of numbers $A=\{1,2,3,4,5\}$. Find $3$ numbers in $A$ that add up to $8$.

**Step 1**: Formulate the problem within its domain 

<div align=left>
    <i>objective</i>:&nbsp; minimize sum over numbers in $A$ <br>
    <i>variables</i>:&nbsp;  correspond to one of the values in $A$ each <br>
    <i>constraints</i>: <ul>
    <li>pick just $3$ numbers</li>
    <li>sum over all numbers is equal to $8$</li>
    </ul>
    
</div>

**Step 2**: Convert objective and constraints to statements with binary variables

We introduce binary variables $x$ for each number in $A$. The variable $x_i$ is equal to $1$ if the corresponding 
number $i$ is part of the solution and $0$ otherwise. Than our objective and constraint read

$$
\textrm{obj:}\hspace{.5cm}\min_{x\in\{0,1\}}\sum_{i=1}^5 i x_i
$$

$$
\textrm{const:}\hspace{.45cm}\sum_{i=1}^5 x_i = 3, \hspace{.5cm}\sum_{i=1}^5 i x_i = 8
$$

**Step 3**: Make the objective and constraints "QUBO appropriate"
- objective is a minimization function
- constraints are satisfied at their minimum value

Lets QUBOfy our objective function.

$$
\textrm{obj:}\hspace{.5cm}\min_{x \in \{0,1\}} \sum_{i=1}^5 i x_i
$$

and for our constrains

$$
\textrm{const:}\hspace{.5cm}\sum_{i=1}^5 x_i - 3,\hspace{.5cm}\sum_{i=1}^5 i x_i - 8
$$

But stop! This is **not correct** because the minimum of these constraints would be the trivial solution $\forall x_i=0$. To avoid this trivial solution we have to **square** the above functions. So the correct constraints are

$$
\textrm{const:}\hspace{.5cm}\left(\sum_{i=1}^5 x_i - 3\right)^2,\hspace{.5cm}\left(\sum_{i=1}^5 i x_i - 8\right)^2
$$


**Step 4**: Combine your objective and constraint 

This gives our final QUBO which reads

$$
   \textrm{QUBO:} \hspace{.5cm} \min_{x \in \{0,1\}} \left[  \left( \sum_{i=1}^5 i x_i - 8 \right) + \gamma  \left(\sum_{i=1}^5 x_i - 3\right)^2 + \gamma \left( \sum_{i=1}^5 i x_i - 8 \right)^2\right]
$$

Here we introduced the parameter $\gamma$ which defines the _penalty strength_ that tunes the impact of our constraint on the overall QUBO. 

Lets run the code to see that it works...

In [ ]:
import dimod
from pyqubo import Binary 

# objective
obj = sum([x*Binary(str(x)) for x in range(1,6)]) - 8

# constraints
const = (sum([Binary(str(x)) for x in range(1,6)]) - 3)**2 + ( sum([x*Binary(str(x)) for x in range(1,6)]) - 8)**2

# penalty strength
gamma = 10

# create QUBO
qubo = obj + gamma*const

In [ ]:
# transform from pyqubo to dimod object
model = qubo.compile()
Q, offset = model.to_qubo()

# create BQM
bqm = dimod.BinaryQuadraticModel.from_qubo(Q, offset=offset)

In [ ]:
from utils.libqubo import show_bqm_graph
show_bqm_graph(bqm)

In [ ]:
exactsolver = dimod.ExactSolver()
sampleset = exactsolver.sample(bqm)

# print the results with lowest energy
sampleset.lowest().to_pandas_dataframe()

Yes! We find the two possible solutions $1+2+5=8$ and $1+3+4=8$. And now on the QPU...

In [ ]:
from dwave.system.samplers import DWaveSampler  

MY_TOKEN = "" 

# list the names of all solvers we can access
sampler = DWaveSampler(token = MY_TOKEN, solver={'qpu': True})
print('Successfully connected to D-Wave Sampler %s.' % sampler.client.get_solver().id)

In [ ]:
from dwave.system.composites import EmbeddingComposite
embedded_sampler = EmbeddingComposite(sampler)

# sample energy space
numruns = 1000
sampleset = embedded_sampler.sample(bqm, num_reads=numruns, chain_strength=1)
print("QPU call complete using", sampleset.info['timing']['qpu_access_time']/1000000, "seconds of QPU time for", numruns, "samples.")

In [ ]:
# show samples
sampleset.to_pandas_dataframe()

Oh what happend here?? That are not the solutions we expect! Well, there are a lot of chain breaks... Maybe we should tune some solver parameters.

In [ ]:
# Alternative: Using QBsolve uses a devide-and-conquer strategie
# devinding the problem over multiple solvers and postselect the lowest energie results
from dwave_qbsolv import QBSolv
sampleset = QBSolv().sample(bqm, solver=sampler, num_reads=1000, chain_strength=1)

# show samples
sampleset.to_pandas_dataframe()

Why is the total number of runs not the number we submitted to the sampler?

# The Penalty approach

The approach we used so far to solve a constraint satisfaction problem, is to map each individual constraint in the to a ‘small’ Ising model or QUBO. This mapping is called a **penalty model**. There are some known penalty models for QUBO constraints.

<img src="images/penalties.jpg" width=800>

# Questions:

- What is the purpose of the Penalty Model? What does it mean in terms of the "energy landscape"?
- What is the meaning of penalty strength? Whats the best value for it?
- What is the meaning of "chain_strength"?
- What do you expect if your optimization problem as near-optimal solutions that are very close to your optimal solutions?
- What are the Hardware constraints to keep in mind when formulating a QUBO for D-Wave?

# Exercise: Graph Partitioning

<img src="images/gp_1.jpg" width=800>

**Step 1**: Formulate the problem within its domain 


**Step 2**: Convert objective and constraints to statements with binary variables


**Step 3**: Make the objective and constraints "QUBO appropriate"


**Step 4**: Combine your objective and constraint 


# Why using D-Wave currently?

Currently D-Wave can work on up to 10k variables and the algorithms make huge progress to solve even larger problems very soon!!

4 coloring problem for Canadian provinces
<img src="images/4color_1.jpg" width=500>

Scaling up using divide-and-concer strategies allows to solve much bigger instances.<br>
4 coloring problem for United States
<br>
<img src="images/4color_2.jpg" width=500>

For coloring problem for United States provinces
<img src="images/4color_4.jpg" width=500>